# GPU-Accelerated Computation using 1D, 2D, and 3D CUDA Kernels
-------------------
## Contents
- **Getting to know your GPU and the CUDA programming model [not graded]**
    - CUDA threads, blocks, and grids, host (CPU) to device (GPU) data movement
    - Writing CUDA kernels
- **1D CUDA Kernels: Vector Addition and 1D Convolution [35 points]**
    - 1D vector addition
        - Simple CUDA [10]
    - 1D convolution
        - Simple CUDA [10]
        - Optimized CUDA [15]
- **2D CUDA Kernels: Matrix Multiplication [20 points]**
    - 2D matrix multiplication
        - Simple CUDA [5]
        - Optimized CUDA [10]
        - Effect of matrix size on performance [5]
- **3D CUDA Kernels: Richardson-Lucy Algorithm for 3D Brain Image Deconvolution [45 points]**
    - 3D Richardson-Lucy
        - Simple CUDA [15]
        - Optimized CUDA [20]
        - Execute RL, compare, and visualize [10]

---------------------
# Getting to know your GPU and the CUDA programming model [not graded]

Let's see the GPU allocated to us using the ```nvidia-smi``` shell command.

In [ ]:
!nvidia-smi

In [ ]:
from numba import cuda
cuda.detect()


Knowing the hardware specification (number of streaming multiprocessors (SMs) and number of cores per SM) helps in choosing a good configuration for running user-defined CUDA kernels. Although selecting an optimal execution configuration needs some analysis, some basic rules are:
- **The number of blocks in the grid should be larger than the number of SMs on the GPU, typically 2 to 4 times larger.**
- **The number of threads per block should be a multiple of 32, typically between 128 and 512.**

CUDA lets you configure the matrix of GPU threads to better fit your task. There is no default value to blockdim and griddim, as the optimal settings is dramatically different for different tasks. In addition to resources posted on Canvas, you can study the ones below:

**CUDA basics:** Watch [this tutorial](https://www.youtube.com/watch?v=kzXjRFL-gjo) (25:25) about CUDA block & grid model.

**Writing CUDA kernels:** https://numba.readthedocs.io/en/stable/cuda/kernels.html. Throughout this MP, keep in mind that CUDA kernels are JIT'ed so the first execution will include numba optimization, compilation, and caching time. To see the benefits of using CUDA kernels, you should time their second runs and not the first.

In [ ]:
from numba import njit, cuda, vectorize, guvectorize, stencil
from numba import prange
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import convolve as scipy_convolve
%matplotlib inline

---------------------
# [Section 1] 1D CUDA Kernels: Vector Addition and 1D Convolution [35 points]

## Traditional vector addition [not graded]

The cell below shows the implementations of vector_addition in native Python (numpy) and cuda numba. For demonstration of what's going on, we will implement the vector addition from scratch by iterating over the two vectors.

![Vector Addition](./resources/vector_addition.png)

In [ ]:
def vector_addition(vector_A:np.ndarray, vector_B:np.ndarray):
    assert vector_A.shape == vector_B.shape
    result = np.empty_like(vector_A)
    for i in range(len(vector_A)):
        result[i] = vector_A[i] + vector_B[i]
    return result

<strong>
Notice that: 

1. There are repeated computational operations on different input data. In this case, the operation is the addition of two numbers at each location of the two vectors.
2. The repeated computations are independent: the result of the addition of the two numbers from ```vector_A``` and ```vector_B``` at location ```i``` depends on nothing but ```vector_A[i]``` and ```vector_B[i]```. 
</strong>

This means vector addition has the potential to be parallelized by having a computing unit process each addition at the same time. In this MP, you will keep identifying such a pattern that lends itself to parallelization. We can "unroll" or "flatten" the for loop to have each iteration of the loop running parallely at the same time. The key is assigning each iteration ```i``` correctly to a GPU thread. We will see how this is done in the following GPU implementation.

![Vector Addition](./resources/vector_addition_parallel.png)

## [Exercise] GPU-Accelerated 1D Vector Addition - Simple CUDA [10 points]

In [ ]:
...
def vector_addition_cuda(...):
    return

In [ ]:
import time

vector_A = np.array([1, 2, 3, 4, 5, 6, 7, 8])
vector_B = np.array([1, 2, 3, 4, 5, 6, 7, 8])

# ---------------------------------------
start_time = time.time()
print("--- [Serial] Starting the timer ---")
serial_result = vector_addition(vector_A, vector_B)
print("Serial result: ", result)
print("--- Done: The execution took %s seconds ---" % (time.time() - start_time))
# ---------------------------------------



# ---------------------------------------
num_blocks = ...
num_threads_per_block = ...

# run and time CUDA kernel

# compare result with serial
assert ...

You may notice a surprising difference between serial and CUDA-based vector addition. We will see how this performance difference changes with the size of data later in the MP.

## [Exercise] 1D Convolution - Simple CUDA [10]

In [ ]:
...
def conv1d_cuda(...):
    return

## [Exercise] 1D Convolution - Optimized CUDA (Using CUDA shared and constant memory) [15]

For this exercise, refer to Prof. Steven Lumetta's slides from the 10/31/22 lecture. He shows us how to take 1D convolutions to the limit of the hardware architecture by making use of tiles, shared memory for data re-use, and constant memory for storing the fixed convolution kernel. His version of the 1D convolution is substantially different from the simple CUDA version: 
- tiled 1D convolution to reuse data loaded into shared memory from global memory (slide 17-23)
- convolution filter goes into constant memory (slide 43)

In [ ]:
...
def faster_conv1d_cuda(...):
    return

In [ ]:
import time
from numba import cuda, float32

# create random data (vector_A) and conv filter (vector_B)
vector_A = ...
vector_B = ...

print("A:", vector_A)
print("B:", vector_B)

scipy_result = ...
print("scipy:", scipy_result)

# configure blocks/threads

# run and time: 1) scipy, 2) simple CUDA, and 3) optimized CUDA kernels

# make sure scipy, simple CUDA, and optimized CUDA all give the same result
assert ...

**Q: What are your observations from the outputs above?**

**Answer:**

----------
# [Section 2] 2D CUDA Kernels: Matrix Multiplication [20 points]

This example was covered in Lab 6. If needed, you can use the following updated code snippets for this section - https://gist.github.com/neerajwagh/40167f82681ea89d65052e2c1bf20f0c

![2-D Matrix Multiplication](./resources/matmul.png)

```
def matmul_serial(mat_A, mat_B):

    assert mat_A.shape[1] == mat_B.shape[0]
    
    output_shape = (mat_A.shape[0], mat_B.shape[1])
    result = np.zeros(output_shape)

    for i in range(output_shape[0]):
        for j in range(output_shape[1]):
            for k in range(mat_B.shape[0]):

                # resulted matrix
                result[i][j] += mat_A[i][k] * mat_B[k][j]

    return result
```

## [Exercise] 2D Matrix Multiplication - Simple CUDA [5]

In [ ]:
...
def matmul_cuda(...):
    return

## [Exercise] 2D Matrix Multiplication - Optimized CUDA [10]

In [ ]:
...
def faster_matmul_cuda(...):
    return

In [ ]:
# create random square matrices of fixed dimension, say 100 
fixed_dim = 100
matrix_A = ...
matrix_B = ...

numpy_result = np.matmul(matrix_A, matrix_B)

# run and time: 1) numpy, 2) simple CUDA, and 3) optimized CUDA kernels

# make sure numpy, simple CUDA, and optimized CUDA all give the same result
assert ...

## [Exercise] Effect of matrix size on performance [5]

Let's see how the performance of the 2D matrix multiplication kernel is affected by increasing sizes of inputs.

In [ ]:
import time

for size in [32, 128, 256, 512, 1024, 2048, 4096]:

    matrix_A = ...
    matrix_B = ...

    if size < 512:
        # ---------------------------------------
        # time serial 
        # ---------------------------------------

    # ---------------------------------------
    # time np.matmul()
    # ---------------------------------------    
        
    # ---------------------------------------
    # time simple and optimized CUDA
    # ---------------------------------------


**Q: What are your observations from the outputs above? How does serial and np.matmul() compare to the CUDA kernels?**

**Answer:**

----------------------------

# [Section 3] 3D CUDA Kernels: Richardson-Lucy Algorithm for 3D Brain Image Deconvolution [45 points]

Let's implement a 3D CUDA kernel for the 3D convolution operation performed as part of the RL algorithm we've seen in the previous MPs.

## [Exercise] 3D Convolution - Simple CUDA [15]

In [ ]:
# define kernel here
...
def convolve(...):
    # single channel: grayscale only
    ...
    return

def convolve3D_gpu(matrix, mask):
    blockdim = ... # number of threads in each direction
    griddim = ... # number of blocks in each direction

    matrix_cont = np.ascontiguousarray(matrix, dtype = matrix.dtype)
    mask_cont = np.ascontiguousarray(mask, dtype = mask.dtype)

    # invoke kernel here
    convolve[...](...); ...

    return result

Use the cell below to test your implementation

In [ ]:
A = np.random.random_sample((240, 240, 155))
B = np.random.random_sample((9, 9, 9))

# time and check correctness of your kernel against scipy convolve
...

## [Exercise] 3D Convolution - Optimized CUDA [20]

In [ ]:
# define kernel here
...
def convolve_optim(...):
    # single channel: grayscale only
    ...
    return

def convolve3D_gpu_optim(matrix, mask):
    blockdim = ... # number of threads in each direction
    griddim = ... # number of blocks in each direction

    matrix_cont = np.ascontiguousarray(matrix, dtype = matrix.dtype)
    mask_cont = np.ascontiguousarray(mask, dtype = mask.dtype)

    # invoke kernel here
    convolve_optim[...](...); ...

    return result

In [ ]:
A = np.random.random_sample((240, 240, 155))
B = np.random.random_sample((9, 9, 9))

# time and check correctness of your kernel against scipy convolve
...

## [Exercise] Setup the RL Algorithm [3]

Use the 3D brain MRI scan downloaded below.

In [ ]:
!wget https://github.com/vb100/Visualize-3D-MRI-Scans-Brain-case/raw/master/data/images/BRATS_001.nii.gz

In [ ]:
import nibabel as nib

image_path = "BRATS_001.nii.gz"
image_obj = nib.load(image_path)
image_data = image_obj.get_fdata()
type(image_data)
print(image_data.shape)
image_data_by_channel = np.array([image_data[:, :, :, i] for i in range(4)])
print(image_data_by_channel.shape)

Copy the 3D RL function from previous MPs below, and drop in the 3D convolution CUDA kernel written above. You will need to two versions - one with simple CUDA and the other with the optimized CUDA kernel.

The following code was released on Canvas earlier - https://gist.github.com/neerajwagh/e438bd7e492fd2bfa4cc28325a73284a

In [ ]:
# use the CUDA kernels instead of scipy convolutions here
def richardson_lucy_3d(...):
    ...
    return im_deconv

You can optionally downsample the image if needed.

In [ ]:
from scipy.ndimage import zoom
RESIZE = False
if RESIZE:
    image_data_by_channel = np.array(
        [zoom(channel, (0.7, 0.7, 0.7)) for channel in image_data_by_channel])

Create a noisy image with a fixed/known PSF that we can then deblur using RL.

In [ ]:
rng = np.random.default_rng()

psf = np.ones((5, 5, 5)) / 125

convolved_by_channel = [scipy_convolve(
    channel_slice, psf, mode = "same") for channel_slice in image_data_by_channel]

noisy_by_channel = convolved_by_channel.copy()

noisy_by_channel = [channel_slice + (rng.poisson(lam=125, size=channel_slice.shape) - 10 / 255)
                    for channel_slice in noisy_by_channel]

print(noisy_by_channel[0].shape)

## [Exercise] Compare execution time: 1) skimage RL, 2) simple CUDA kernel, and 3) optimized CUDA kernel [4]

In [ ]:
NUM_ITERATIONS_RL = ...

# skimage version can be found at https://scikit-image.org/docs/stable/api/skimage.restoration.html#skimage.restoration.richardson_lucy 

...

## [Exercise] Visualization - Compare the blurred image and deconvolved image for 1) skimage RL, 2) simple CUDA kernel, and 3) optimized CUDA kernel [3]

Note - this should be a 3x2 panel plot.

In [ ]:
...